In [1]:
import numpy as np
import random,copy,time
import pygame as pg

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def weights_classifier(structure,chromosome):
    weights_list=[]
    for i in range(0,len(structure)-2):
        if i == 0:
            a=0
            b=structure[i]*structure[i+1]
        else:
            a=b
            b=a+(structure[i]*structure[i+1])
        r=structure[i]
        c=structure[i+1]
        w=np.reshape(np.matrix(chromosome[a:b]),(r,c))
        weights_list.append(w)
    return weights_list

def sigmoid(x):
    return 1/(1+np.exp(-x))

def feed_forward(structure,chromosome,train_input):
    weights_list=weights_classifier(structure,chromosome)
    for i in range(len(weights_list)):
        if i==0:
            output=sigmoid(np.dot(train_input,weights_list[i]))
        else:
            output=sigmoid(np.dot(output,weights_list[i]))     
    return output
 
def playing(structure,chromosome,train_input):
    decision=feed_forward(structure,chromosome,train_input)
    return np.array(decision)

In [3]:
def visionApple(snake,apple):
    vision_apple=[0, 0, 0, 0, 0, 0, 0, 0]
    n=snake[0][1]
    s=snake[0][1]
    w=snake[0][0]
    e=snake[0][0]

    for i in range(10):

        n-=60
        s+=60
        w-=60
        e+=60

        if apple[1]==n and apple[0]==snake[0][0]:
            vision_apple[0]=1

        if apple[1]==s and apple[0]==snake[0][0]:
            vision_apple[1]=1  

        if apple[0]==w and apple[1]==snake[0][1]:
            vision_apple[2]=1      

        if apple[0]==e and apple[1]==snake[0][1]:
            vision_apple[3]=1        

        if apple[1]==n and apple[0]==w:
            vision_apple[4]=1    

        if apple[1]==s and apple[0]==w:
            vision_apple[5]=1    

        if apple[1]==n and apple[0]==e:
            vision_apple[6]=1 

        if apple[1]==s and apple[0]==e:
            vision_apple[7]=1
            
    return vision_apple

def visionBody(snake,apple):
    vision_body=[0, 0, 0, 0, 0, 0, 0, 0]

    for i in range(1,len(snake)):
        
        n=snake[0][1]
        s=snake[0][1]
        w=snake[0][0]
        e=snake[0][0]
        
        for _ in range(10):

            n-=60
            s+=60
            w-=60
            e+=60

            if snake[i][1]==n and snake[i][0]==snake[0][0]:
                vision_body[0]=1

            if snake[i][1]==s and snake[i][0]==snake[0][0]:
                vision_body[1]=1  

            if snake[i][0]==w and snake[i][1]==snake[0][1]:
                vision_body[2]=1      

            if snake[i][0]==e and snake[i][1]==snake[0][1]:
                vision_body[3]=1        

            if snake[i][1]==n and snake[i][0]==w:
                vision_body[4]=1    

            if snake[i][1]==s and snake[i][0]==w:
                vision_body[5]=1    

            if snake[i][1]==n and snake[i][0]==e:
                vision_body[6]=1 

            if snake[i][1]==s and snake[i][0]==e:
                vision_body[7]=1
            
    return vision_body

def visionWall(snake):
    #            nw,sw,ww,ew
    vision_wall=[0, 0, 0, 0]
    
    vision_wall[0]=int((snake[0][1]/60))
    vision_wall[1]=int(((600-snake[0][1])/60)-1)
    vision_wall[2]=int((snake[0][0]/60))
    vision_wall[3]=int(((600-snake[0][0])/60)-1)
    
    return vision_wall

def move_d(decision):
    
    move=[0,0,0,0]
    j=np.amax(decision)
    k=np.where(decision==j)
    i=k[1][0]
    #print('\nj= ',j,' k= ',k,' i= ',i)
    move[i]=1
    
    return move

In [4]:
def game(structure,chromosome):
    
    frame_w=600
    frame_h=600
    
    pg.init()
    screen = pg.display.set_mode((frame_w,frame_h))

    white=[255,255,255]
    red=[255,0,0]
    black=[0,0,0]
    green=[0,255,0]
    
    running=True
    apple_present=False

    score=0
    steps=0
    apple_steps=200

    x=[]
    y=[]

    for i in range(0,frame_w,60):
        x.append(i)
        y.append(i)

    grid=[]
    for i in range(len(x)):
        for j in range(len(y)):
            grid.append((x[i],y[j]))

    snake_head=grid[55]
    snake=[snake_head]
    
    while running:
        
        pg.event.get()
        
        grid_c=copy.deepcopy(grid)

        if apple_present==False:
            for body in snake:
                if body in grid:
                    grid.remove(body)
            apple=random.choice(grid)
            apple_present=True

        vision_apple=visionApple(snake,apple)

        #print('\nApple= ',vision_apple)

        #print('snake= ',snake,' apple= ',apple,' Score= ',score,' steps= ',steps)

        if steps==0:
            vision_body=[0, 0, 0, 0, 0, 0, 0, 0]
            vision_wall=[0, 0, 0, 0]
            head_direction=[0, 0, 0, 0]
            tail_direction=[0, 0, 0, 0]

        nn_input=np.concatenate([vision_apple,vision_body,vision_wall,head_direction,tail_direction])

        ###call nn
        decision=playing(structure,chromosome,nn_input)
        #### call move
        move=move_d(decision)
        #print(decision)
        

        snake_c=copy.deepcopy(snake)

        if move ==[1,0,0,0]:
            snake[0]=(snake[0][0],snake[0][1]-60)
        if move ==[0,1,0,0]:
            snake[0]=(snake[0][0],snake[0][1]+60)
        if move ==[0,0,1,0]:
            snake[0]=(snake[0][0]-60,snake[0][1])
        if move ==[0,0,0,1]:
            snake[0]=(snake[0][0]+60,snake[0][1])
            
            
        for i in range(1,len(snake)):

            if snake[i][0]==snake[0][0] and snake[i][1]==snake[0][1]:
                #print('snake= ',snake,' apple= ',apple,' Score= ',score,' steps= ',steps)
                #print('Body Collision')
                running=False
                

        for i in range(1,len(snake)):
            snake[i]=snake_c[i-1]

        if snake[0]==apple:
            apple_present=False
            snake.append(snake_c[-1])
            #snake_c=copy.deepcopy(snake)
            score+=1
            apple_steps=200
            #print('Apple Gone')

        if len(snake)==1:
            vision_body=[0, 0, 0, 0, 0, 0, 0, 0]
        elif len(snake)>1:
            vision_body=visionBody(snake,apple)
        #print('Body= ',vision_body)

        if snake[0][0] < 0 or snake[0][0] > 540 or snake[0][1] < 0 or snake[0][1] > 540 :
            #print('Wall Collision')
            running=False


        if len(snake) != 1:        
            if (snake_c[-1][1]-snake[-1][1])<0:
                tail_direction=[0,1,0,0]
            if (snake_c[-1][1]-snake[-1][1])>0:
                tail_direction=[1,0,0,0]
            if (snake_c[-1][0]-snake[-1][0])<0:
                tail_direction=[0,0,0,1]
            if (snake_c[-1][0]-snake[-1][0])>0:
                tail_direction=[0,0,1,0]
        else:
            tail_direction=move

        head_direction=move

        vision_wall=visionWall(snake)
        
        steps+=1
        apple_steps -= 1
        if apple_steps == 0:
            running=False
        #print(running)
        screen.fill(white)

        for i in range(len(grid_c)):
            g_grid=pg.Rect(grid_c[i][0],grid_c[i][1],60,60)
            pg.draw.rect(screen,black,g_grid,1)

        g_apple=pg.Rect(apple[0],apple[1],60,60)
        pg.draw.rect(screen,green,g_apple)

        for i in range(len(snake)):
            if i==0:
                color=red
            else:
                color=black
            g_snake=pg.Rect(snake[i][0],snake[i][1],60,60)
            pg.draw.rect(screen,color,g_snake)

        pg.display.flip()
        
        time.sleep(0.2)
    return score,steps

In [ ]:
structure=[28,14,7,4,0]
for j in range(1,1000):
    
    file=open('weights\gen_'+str(j)+'.txt','r')
    weights=file.read()
    file.close()

    refined_w=weights.split(',')
    chromosome=[None]*len(refined_w)
    
    for i in range(len(refined_w)):
        if i==0:
            chromosome[i]=(float(refined_w[i].split('[')[1]))
        elif i==len(refined_w)-1:
            chromosome[i]=(float(refined_w[i].split(']')[0]))
        else:
            chromosome[i]=(float(refined_w[i]))
            
    score,steps=game(structure,chromosome)  
    print('Gen= ',j,'-----Score= ',score)

Gen=  1 -----Score=  0
Gen=  2 -----Score=  0
Gen=  3 -----Score=  0
Gen=  4 -----Score=  0
Gen=  5 -----Score=  1
Gen=  6 -----Score=  0
Gen=  7 -----Score=  0
Gen=  8 -----Score=  0
Gen=  9 -----Score=  0
Gen=  10 -----Score=  0
Gen=  11 -----Score=  0
Gen=  12 -----Score=  0
Gen=  13 -----Score=  0
Gen=  14 -----Score=  0
Gen=  15 -----Score=  0
Gen=  16 -----Score=  0
Gen=  17 -----Score=  0
Gen=  18 -----Score=  0
Gen=  19 -----Score=  0
Gen=  20 -----Score=  1
Gen=  21 -----Score=  0
Gen=  22 -----Score=  1
Gen=  23 -----Score=  0
Gen=  24 -----Score=  0
Gen=  25 -----Score=  0
Gen=  26 -----Score=  0
Gen=  27 -----Score=  1
Gen=  28 -----Score=  0
Gen=  29 -----Score=  0
Gen=  30 -----Score=  0
Gen=  31 -----Score=  0
Gen=  32 -----Score=  0
Gen=  33 -----Score=  0
Gen=  34 -----Score=  0
Gen=  35 -----Score=  0
Gen=  36 -----Score=  0
Gen=  37 -----Score=  0
Gen=  38 -----Score=  0
Gen=  39 -----Score=  0
Gen=  40 -----Score=  0
Gen=  41 -----Score=  0
Gen=  42 -----Score=  0
G

Gen=  334 -----Score=  0
Gen=  335 -----Score=  1
Gen=  336 -----Score=  0
Gen=  337 -----Score=  0
Gen=  338 -----Score=  1
Gen=  339 -----Score=  0
Gen=  340 -----Score=  0
Gen=  341 -----Score=  0
Gen=  342 -----Score=  0
Gen=  343 -----Score=  0
Gen=  344 -----Score=  1
Gen=  345 -----Score=  1
Gen=  346 -----Score=  0
Gen=  347 -----Score=  0
Gen=  348 -----Score=  0
Gen=  349 -----Score=  0
Gen=  350 -----Score=  0
Gen=  351 -----Score=  0
Gen=  352 -----Score=  0
Gen=  353 -----Score=  0
Gen=  354 -----Score=  0
Gen=  355 -----Score=  0
Gen=  356 -----Score=  0
Gen=  357 -----Score=  1
Gen=  358 -----Score=  0
Gen=  359 -----Score=  0
Gen=  360 -----Score=  0
Gen=  361 -----Score=  1
Gen=  362 -----Score=  0
Gen=  363 -----Score=  0
Gen=  364 -----Score=  0
Gen=  365 -----Score=  0
Gen=  366 -----Score=  1
Gen=  367 -----Score=  0
Gen=  368 -----Score=  0
Gen=  369 -----Score=  0
Gen=  370 -----Score=  0
Gen=  371 -----Score=  1
Gen=  372 -----Score=  1
Gen=  373 -----Score=  0
